# Neural Beagle 7b on Kaggle with LLama-cpp

![](https://i.imgur.com/89ZAKcn.png)

# Descarga del modelo cuantizado desde huggingface

In [ ]:
from huggingface_hub import hf_hub_download

repo_url = 'TheBloke/NeuralBeagle14-7B-GGUF'
model_filename = 'neuralbeagle14-7b.Q4_0.gguf'

destination_dir = '/kaggle/working/'

model_path = hf_hub_download(repo_url, filename=model_filename, cache_dir=destination_dir)

print(f"Modelo descargado en(Download in this path): {model_path}")

# Aca en teoria se habilita el proceso de cuda (gpu) pero la inferencia se realzia con CPU Be cause? I don´t know

In [2]:
# Installation of llama-cpp

!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1
!pip install -v llama-cpp-python

Using pip 23.2.1 from /opt/conda/lib/python3.10/site-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 47.8 MB/s eta 0:00:0000:010:01
  Running command pip subprocess to install build dependencies
    Obtaining dependency information for scikit-build-core[pyproject]>=0.5.1 from https://files.pythonhosted.org/packages/0c/5b/73dc7944ef0fdbe97626b40525f1f9ca2547d7c5229b358d45357ff62209/scikit_build_core-0.8.0-py3-none-any.whl.metadata
    Obtaining dependency information for exceptiongroup from https://files.pythonhosted.org/packages/b8/9a/5028fd52db10e600f1c4674441b968cf2ea4959085bfb5b99fb1250e5f68/exceptiongroup-1.2.0-py3-none-any.whl.metadata
    Obtaining dependency information for packaging>=20.9 from https://files.pythonhosted.org/packages/ec/1a/610693ac4ee14fcdf2d9bf3c493370e4f2ef7ae2e19217d7a237ff42367d/packaging-23.2-py3-none-any.whl.metadata
    Obtaining dependency information for pathspec>=0.10.1 from https://files.pythonhosted.org/packages/

# Se instala LangChain, tambien te podes loguear para acceder desde tu perfil, pero es opccional

In [5]:
# LangChain
!pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/23/9f/a78357793c96ae5b53b5a31a891ed2fe3b02dc1a11a705dd14da67932c42/langchain-0.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.14 from https://files.pythonhosted.org/packages/57/00/a798f8124db57eb9e20fe31dc7561e15e9c4607281cddaa4db49f93d7111/langchain_community-0.0.16-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.16 from https://files.pythonhosted.org/packages/e6/7f/2c5006e2292bbcc9fc7cfaac407954c00fc8c9f5afd4e62c17adc0ba1790/langchain_core-0.1.17-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1,>=0.0.83 from https://files.pythonhosted.org/packages/61/f2/45e

# Aca cargamos el GPU de adorno porque llama-cpp por el motivo x (no hay respuestas, solo preguntas en todos lados) no es compatible con el GPU de kaggle, mas alla de eso la inferencia no es tan lenta, con GPU debe volar!

In [14]:
!nvidia-smi
!nvcc --version
import torch

if not torch.cuda.is_available():
    raise RuntimeError()

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

model_path = '/kaggle/working/models--TheBloke--NeuralBeagle14-7B-GGUF/snapshots/8952d8c1de658040fab3a45c9af426d468cfadb2/neuralbeagle14-7b.Q4_0.gguf'  # Reemplaza esto con la ruta de tu modelao
llm_kwargs = {
    "temperature": 0.75,
    "max_tokens": 5000,
    "top_p": 1,
    "n_gpu_layers": 40,
    "n_batch": 512,
    "callback_manager": CallbackManager([StreamingStdOutCallbackHandler()]),
    "verbose": True,
}

if not os.path.exists(model_path):
    raise FileNotFoundError(model_path)

llm = LlamaCpp(
    model_path=model_path,
    **llm_kwargs,
)

template = "Question: {question}\n\nAnswer: Let's work this out in a step by step way to be sure we have the right answer."

llm_chain = PromptTemplate(template=template, input_variables=["question"]) | llm

question = "Your question here"
answer = llm_chain.invoke(dict(question=question))


Mon Jan 29 21:05:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /kaggle/working/models--TheBloke--NeuralBeagle14-7B-GGUF/snapshots/8952d8c1de658040fab3a45c9af426d468cfadb2/neuralbeagle14-7b.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mlabonne_neuralbeagle14-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.

 First, let's make sure we understand that 70% of a number is equal to 56. This can be expressed as an equation where x is the unknown number:

0.7x = 56

To solve for x, we divide both sides of the equation by 0.7 (which is equivalent to multiplying both sides by its reciprocal, which is 1/0.7 or approximately 1.428):

x = 56 / 0.7
x = 80

Thus, if 70% of a number is equal to 56, the actual number is 80.


llama_print_timings:        load time =   10407.97 ms
llama_print_timings:      sample time =      75.05 ms /   146 runs   (    0.51 ms per token,  1945.42 tokens per second)
llama_print_timings: prompt eval time =   10407.85 ms /    32 tokens (  325.25 ms per token,     3.07 tokens per second)
llama_print_timings:        eval time =   58076.87 ms /   145 runs   (  400.53 ms per token,     2.50 tokens per second)
llama_print_timings:       total time =   69306.72 ms /   177 tokens


# Reduccion de código para hacer la inferenccia

In [18]:
template = "Question: {question}\n\nAnswer: Quiero que hables siempre en español, y respondas como un asistente"

llm_chain = PromptTemplate(template=template, input_variables=["question"]) | llm

question = "como crees que va a evolucionas los grandes modelos de lenguajes de la IA?"
answer = llm_chain.invoke(dict(question=question))

Llama.generate: prefix-match hit


 generalizado. A medida que avanza la investigación en inteligencia artificial (IA) y aprendizaje automático, se espera que los grandes modelos de lenguajes de IA sigan evolucionando en varias áreas importantes:

1. **Capacidad**: Actualmente, los modelos más grandes como GPT-3 y BERT tienen capacidades impresionantes pero aún no alcanzan el nivel de un humano en muchas tareas. Se espera que los modelos futuros sean capaces de procesar y entender información con mayor velocidad, precisión y eficiencia.

2. **Especialización**: Actualmente, los modelos de lenguaje se entrenan sobre un conjunto amplio de datos, lo que significa que no son especialistas en un área específica. Se espera que los futuros modelos estén diseñados para ser especialistas en áreas concretas, como medicina, ingeniería, financiero, etc., y puedan proporcionar respuestas más acertadas y detalladas en esos campos.

3. **Ethics, Bias and Privacy**: Actualmente, existen preocupaciones sobre la ética, sesgos y privacida


llama_print_timings:        load time =   10407.97 ms
llama_print_timings:      sample time =     264.07 ms /   459 runs   (    0.58 ms per token,  1738.20 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  187922.16 ms /   459 runs   (  409.42 ms per token,     2.44 tokens per second)
llama_print_timings:       total time =  190827.09 ms /   460 tokens
